In [1]:
import hopsworks
import os
import great_expectations as ge
from great_expectations.core import ExpectationSuite, ExpectationConfiguration

import math
import pandas as pd

In [2]:
# Connect to the Hopsworks Feature Store
project = hopsworks.login()
fs = project.get_feature_store()

2024-10-17 14:35:50,759 INFO: Python Engine initialized.

Logged in to project, explore it here https://demo.hops.works/p/123


In [3]:
# Read the historical transaction data
transactions_pdf = pd.read_csv(f"{os.environ['PROJECT_PATH']}/Jupyter/RawData/historical_transactions.csv", parse_dates=['datetime'])

In [4]:
transactions_pdf = transactions_pdf[["tid", "datetime", "cc_num", "category", "amount", "latitude", 'longitude', 'city', 'fraud_label']]

# cc_num is more of an account_id rather than a proper credit card number, rename the column to avoid confusion
transactions_pdf.rename(columns={'cc_num': 'account_id', 'city': 'transaction_city'}, inplace=True) 

In [5]:
transactions_pdf

,tid,datetime,account_id,category,amount,latitude,longitude,transaction_city,fraud_label
0,b55221e7b96145f4dcca6a7145f63902,2024-04-18 22:27:46,1766230b4f7602f856ffc82ef0a0ffd1,Grocery,78.31,36.025060,-86.779170,Brentwood Estates,0
1,2009c06f5ee87fd5737ea7fe5906ffdd,2024-04-18 22:29:46,c30018ba5ca059426ca40184e3b4d833,Grocery,77.14,33.410120,-91.061770,Greenville,0
2,74b9a4c0340ecf36c5d7a6a3b2041348,2024-04-18 22:34:03,818951f6e43105a228bddc51a19d8e28,Grocery,13.92,33.036990,-117.291980,Encinitas,0
3,0dea360335b962a5dbd85aa4afeb8727,2024-04-18 22:38:17,4fd1d53696a07c807e56c6af5c195430,Grocery,33.05,29.845760,-90.106740,Estelle,0
4,08709a9837af068692fb0adaa55c0817,2024-04-18 22:48:09,3d66cf2dc257be18dbed42327f64f753,Clothing,65.18,40.605380,-73.755130,Far Rockaway,0
...,...,...,...,...,...,...,...,...,...
71317,a1a2199db6841f4db11af72335ede20c,2024-08-16 21:49:18,39ccf08351228fbc6073640bdfa58016,Cash Withdrawal,704.83,40.718469,-73.926426,Greenpoint,0
71318,cd2d9132a9923dfdf5919686c1a159e8,2024-08-12 21:49:18,39ccf08351228fbc6073640bdfa58016,Cash Withdrawal,50.53,40.723734,-73.925561,Greenpoint,0
71319,be6db07b40104497d568f85365d1b41e,2024-08-08 21:49:18,39ccf08351228fbc6073640bdfa58016,Cash Withdrawal,61.86,40.722977,-73.926932,Greenpoint,0
71320,185138197b19a11c086ccf752cd60b74,2024-08-04 21:49:18,39ccf08351228fbc6073640bdfa58016,Cash Withdrawal,412.33,40.713166,-73.921838,Greenpoint,0


In [6]:
transactions_pdf['datetime'].min()

Timestamp('2024-02-10 14:51:56')

In [7]:
transactions_pdf['datetime'].max()

Timestamp('2024-10-15 22:08:21')

In [8]:
# Sort the dataframe by 'account_id' and 'datetime' to ensure transactions are ordered by time for each account
transactions_pdf = transactions_pdf.sort_values(by=['account_id', 'datetime']).reset_index(drop=True)

# Shift the 'date_time', 'latitude', and 'longitude' columns by 1 to get the previous transaction details
transactions_pdf['last_transaction_datetime'] = transactions_pdf.groupby('account_id')['datetime'].shift(1)
transactions_pdf['last_latitude'] = transactions_pdf.groupby('account_id')['latitude'].shift(1)
transactions_pdf['last_longitude'] = transactions_pdf.groupby('account_id')['longitude'].shift(1)

In [9]:
@hopsworks.udf(return_type=float, drop=["latitude", "longitude", "last_latitude", "last_longitude"])
def loc_delta_t_minus_1(latitude: pd.Series, longitude: pd.Series, last_latitude: pd.Series, last_longitude: pd.Series) -> pd.Series:
    """
    Calculate the Haversine distance between two latitude/longitude points in kilometers for pandas Series inputs.
    
    Parameters:
        latitude (pd.Series): Series of latitudes for the first set of points in decimal degrees.
        longitude (pd.Series): Series of longitudes for the first set of points in decimal degrees.
        last_latitude (pd.Series): Series of latitudes for the second set of points in decimal degrees.
        last_longitude (pd.Series): Series of longitudes for the second set of points in decimal degrees.
        
    Returns:
        pd.Series: Series of Haversine distances in kilometers.
    """

    import pandas as pd
    from math import radians, sin, cos, sqrt, atan2
    
    # Radius of the Earth in kilometers
    R = 6371.0
    
    # Convert latitude and longitude from degrees to radians
    latitude_rad = latitude.apply(radians)
    longitude_rad = longitude.apply(radians)
    last_latitude_rad = last_latitude.apply(radians)
    last_longitude_rad = last_longitude.apply(radians)
    
    # Compute differences between coordinates
    dlat = last_latitude_rad - latitude_rad
    dlon = last_longitude_rad - longitude_rad
    
    # Haversine formula
    a = (dlat / 2).apply(sin)**2 + latitude_rad.apply(cos) * last_latitude_rad.apply(cos) * (dlon / 2).apply(sin)**2
    c = 2 * a.apply(sqrt).apply(lambda x: atan2(x, sqrt(1 - x)))
    
    # Distance in kilometers
    distance = R * c
    return distance

2024-10-17 14:35:58,334 WARNING: UserWarning: Cannot extract imported dependencies for the UDF from the module in which it is defined. Please make sure to import all dependencies for the UDF inside the function.



In [10]:
@hopsworks.udf(return_type=float, drop=["last_transaction_datetime"])
def time_delta_t_minus_1(datetime, last_transaction_datetime):
    """
    Calculate the difference between two dates in days.
    
    Parameters:
        date_time (datetime): The date of the current transaction as a datetime object .
        last_transaction_datetime (datetime): The date of the previous transaction as a datetime object.    
        
    Returns:
        float: The absolute difference in days between the two dates.
    """
    return (datetime - last_transaction_datetime).dt.days

In [11]:
@hopsworks.udf(return_type=float, drop=["city", "transaction_city"])
def is_outside_city(city, transaction_city):
    return (city != transaction_city).astype(int)

In [13]:
# Compute feature to determine whether or not a transaction was made outside the city of residence of the user
# Retrieve user profiles information
profiles_fg = fs.get_feature_group("profiles", version=1)
profiles_pdf = profiles_fg.select(['account_id', 'city']).read()

# Join with transaction dataframe
transactions_pdf = transactions_pdf.merge(profiles_pdf, on='account_id')

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.14s) 


In [14]:
transactions_pdf

,tid,datetime,account_id,category,amount,latitude,longitude,transaction_city,fraud_label,last_transaction_datetime,last_latitude,last_longitude,city
0,c7183c7e946acd9eae1134f38aa20766,2024-04-19 01:08:30,0012ad418fd2dd753e0f6bc5a7471c98,Grocery,54.04,38.17492,-122.26080,American Canyon,0,NaT,NaN,NaN,Searcy
1,309d56538d8fdafd01e6b32ab169eb4d,2024-04-19 18:46:02,0012ad418fd2dd753e0f6bc5a7471c98,Health/Beauty,33.19,47.92526,-97.03285,Grand Forks,0,2024-04-19 01:08:30,38.17492,-122.26080,Searcy
2,9a9eed25acaf4f3b49564afe128a21b7,2024-04-20 10:56:48,0012ad418fd2dd753e0f6bc5a7471c98,Grocery,37.53,42.73920,-84.62081,Waverly,0,2024-04-19 18:46:02,47.92526,-97.03285,Searcy
3,f138388fa5c911d0a668ec7aa41cce75,2024-04-21 20:18:44,0012ad418fd2dd753e0f6bc5a7471c98,Health/Beauty,61.95,26.68451,-80.66756,Belle Glade,0,2024-04-20 10:56:48,42.73920,-84.62081,Searcy
4,7d754d42c447e7d61ee5cb14703bef46,2024-04-26 09:07:06,0012ad418fd2dd753e0f6bc5a7471c98,Health/Beauty,630.99,31.12406,-97.90308,Copperas Cove,0,2024-04-21 20:18:44,26.68451,-80.66756,Searcy
...,...,...,...,...,...,...,...,...,...,...,...,...,...
71317,c26f46d835ab2e530643a45dcb1547b2,2024-10-04 09:44:59,ffb64785ff2fd0b1288b0f1ce6b9bfd6,Restaurant/Cafeteria,2.58,40.55760,-74.28459,Woodbridge,0,2024-09-21 21:47:35,36.20829,-115.98391,Searcy
71318,86515b66e76317adba05745ea6437d88,2024-10-04 23:18:08,ffb64785ff2fd0b1288b0f1ce6b9bfd6,Grocery,9.10,37.52487,-77.55777,Bon Air,0,2024-10-04 09:44:59,40.55760,-74.28459,Searcy
71319,132d608582915aa6d6135a9f747c4178,2024-10-11 10:44:55,ffb64785ff2fd0b1288b0f1ce6b9bfd6,Holliday/Travel,91.63,32.42067,-104.22884,Carlsbad,0,2024-10-04 23:18:08,37.52487,-77.55777,Searcy
71320,b9079742a6c8f7532be23fb5c88a2520,2024-10-13 16:03:35,ffb64785ff2fd0b1288b0f1ce6b9bfd6,Domestic Transport,97.47,45.04080,-93.26300,Columbia Heights,0,2024-10-11 10:44:55,32.42067,-104.22884,Searcy


In [15]:
# Convert the 'trans_df' DataFrame to a Great Expectations DataFrame
ge_trans_df = ge.from_pandas(transactions_pdf)

# Retrieve the expectation suite associated with the ge DataFrame
expectation_suite = ge_trans_df.get_expectation_suite()

# Set the expectation suite name to "transactions_suite"
expectation_suite.expectation_suite_name = "transactions_suite"

# Check binary fraud_label column to be in set [0,1]
expectation_suite.add_expectation(
    ExpectationConfiguration(
        expectation_type="expect_column_distinct_values_to_be_in_set",
        kwargs={
            "column": "fraud_label",
            "value_set": [0, 1],
        }
    )
)

# Check amount column to be not negative
expectation_suite.add_expectation(
    ExpectationConfiguration(
        expectation_type="expect_column_values_to_be_between",
        kwargs={
            "column": "amount",
            "min_value": 0.0,
        }
    )
)

2024-10-17 14:38:12,967 INFO: 	0 expectation(s) included in expectation_suite.


{"expectation_type": "expect_column_values_to_be_between", "kwargs": {"column": "amount", "min_value": 0.0}, "meta": {}}

In [16]:
# Create the feature group metadata
transactions_fg = fs.get_or_create_feature_group(
    name="transactions",
    version=1,
    description="Credit card transaction data",
    primary_key=['tid'],
    event_time='datetime',
    online_enabled=True,
    expectation_suite=expectation_suite,
    statistics_config={'histograms': True, 'correlations': True},
    transformation_functions=[loc_delta_t_minus_1, time_delta_t_minus_1, is_outside_city],
    parents=[profiles_fg]
)

In [17]:
# Set data validation option to Strict
transactions_fg.expectation_suite.validation_ingestion_policy = "STRICT"

In [18]:
# Insert data into feature group
transactions_fg.insert(transactions_pdf)

Feature Group created successfully, explore it at 
https://demo.hops.works/p/123/fs/68/fg/1076
2024-10-17 14:38:25,149 INFO: 	2 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://demo.hops.works/p/123/fs/68/fg/1076


Uploading Dataframe: 100.00% |██████████| Rows 71322/71322 | Elapsed Time: 00:04 | Remaining Time: 00:00


Launching job: transactions_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://demo.hops.works/p/123/jobs/named/transactions_1_offline_fg_materialization/executions


(Job('transactions_1_offline_fg_materialization', 'SPARK'),
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_values_to_be_between",
         "kwargs": {
           "column": "amount",
           "min_value": 0.0
         },
         "meta": {
           "expectationId": 1048
         }
       },
       "result": {
         "element_count": 71322,
         "missing_count": 0,
         "missing_percent": 0.0,
         "unexpected_count": 0,
         "unexpected_percent": 0.0,
         "unexpected_percent_total": 0.0,
         "unexpected_percent_nonmissing": 0.0,
         "partial_unexpected_list": []
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2024-10-17T02:38:25.000149Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     },
     {
   

In [19]:
# Update feature descriptions
feature_descriptions = [
    {"name": "tid", "description": "Transaction id"},
    {"name": "datetime", "description": "Transaction time"},
    {"name": "account_id", "description": "Account performing the transaction"},
    {"name": "amount", "description": "Dollar amount of the transaction"},
    {"name": "fraud_label", "description": "Whether the transaction was fraudulent or not"},
    {"name": "loc_delta_t_minus_1", "description": "Location of previous transaction"},
    {"name": "time_delta_t_minus_1", "description": "Time of previous transaction"},
    {"name": "is_outside_city", "description": "Whether the transaction was done outside the city of residence"},
]

for desc in feature_descriptions: 
    transactions_fg.update_feature_description(desc["name"], desc["description"])